## 1. 数据读取与预览(老规矩)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [2]:
df=pd.read_csv("./dataset/个人收入水平调查分析.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 7 columns):
年龄        32561 non-null int64
受教育时间     32561 non-null int64
性别        32561 non-null object
资产净增      32561 non-null int64
资产损失      32561 non-null int64
一周工作时间    32561 non-null int64
收入水平      32561 non-null object
dtypes: int64(5), object(2)
memory usage: 1.7+ MB


In [3]:
df.describe()

,年龄,受教育时间,资产净增,资产损失,一周工作时间
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


## 2. 数据预处理

In [4]:
def one_or_zero(x):
  if x=="Male" or ">" in x:
    return 1 
  else :
    return 0
  
df["性别"]=df["性别"].apply(one_or_zero)
df["收入水平"]=df["收入水平"].apply(one_or_zero)

In [5]:
data=df.copy()
# data["性别"].unique()
# data["收入水平"].unique()
data.head()

,年龄,受教育时间,性别,资产净增,资产损失,一周工作时间,收入水平
0,39,13,1,2174,0,40,0
1,50,13,1,0,0,13,0
2,38,9,1,0,0,40,0
3,53,7,1,0,0,40,0
4,28,13,0,0,0,40,0


### 2.1 调试阶段(数据分割)

In [7]:
m, n = data.shape
da=np.array(data)
data.iloc[:,-1]
data.iloc[:,:-1]

# 数据集分割
data_mat=np.array(data.iloc[:20000,:-1])
class_labels=np.array(data.iloc[:20000,-1])

data_mat_ce=np.array(data.iloc[20000:,:-1])
class_label_ce=np.array(data.iloc[20000:,-1])
type(class_labels)


print(len(class_label_ce))
for i in class_label_ce[:3]:
    print(i)
print(len(data_mat_ce))   


12561
0
1
0
12561


## 3. 回归系数计算
### 3.1  随机梯度上升算法（随机化）

In [8]:
def sigmoid(x):
    # 这里其实非常有必要解释一下，会出现的错误 RuntimeWarning: overflow encountered in exp
    # 这个错误在学习阶段虽然可以忽略，但是我们至少应该知道为什么
    # 这里是因为我们输入的有的 x 实在是太小了，比如 -6000之类的，那么计算一个数字 np.exp(6000)这个结果太大了，没法表示，所以就溢出了
    # 如果是计算 np.exp（-6000），这样虽然也会溢出，但是这是下溢，就是表示成零
    # 去网上搜了很多方法，比如 使用bigfloat这个库（我竟然没有安装成功，就不尝试了，反正应该是有用的
    return 1.0 / (1 + np.exp(-x))
def stoc_grad_ascent1(data_mat, class_labels, num_iter=150):
    """
    改进版的随机梯度上升，使用随机的一个样本来更新回归系数
    :param data_mat: 输入数据的数据特征（除去最后一列）,ndarray
    :param class_labels: 输入数据的类别标签（最后一列数据
    :param num_iter: 迭代次数
    :return: 得到的最佳回归系数
    """
    m, n = np.shape(data_mat)
    # 初始化权重系数
    weights = np.ones(n)
    for j in range(num_iter):
        # 这里必须要用list，不然后面的del没法使用
        data_index = list(range(m))
        for i in range(m):
            # i和j的不断增大，导致alpha的值不断减少，但是不为0
            alpha = 4 / (1.0 + j + i) + 0.01
            # 随机产生一个 0～len()之间的一个值
            # random.uniform(x, y) 方法将随机生成下一个实数，它在[x,y]范围内,x是这个范围内的最小值，y是这个范围内的最大值。
            rand_index = int(np.random.uniform(0, len(data_index)))
            
            # 使用随机值索引
            h = sigmoid(np.sum(data_mat[data_index[rand_index]] * weights))
            
            # 计算差值
            error = class_labels[data_index[rand_index]] - h
            
            # 重新配置权值
            weights = weights + alpha * error * data_mat[data_index[rand_index]]
            
            #del(data_index[rand_index])
        #print(pd.DataFrame(weights).head(5))
        #print(pd.DataFrame(data_mat[data_index[rand_index]]).head(5))
        #rint(h)
    return weights
#a=stoc_grad_ascent1(data_mat, class_labels, num_iter=10)
#print(a)

In [9]:
#  循环遍历计算
def classify_vector(in_x, weights):
    """
    最终的分类函数，根据回归系数和特征向量来计算 Sigmoid 的值，大于0.5函数返回1，否则返回0
    :param in_x: 特征向量，features
    :param weights: 根据梯度下降/随机梯度下降 计算得到的回归系数
    :return: 
    """
    # print(np.sum(in_x * weights))
    prob = sigmoid(np.sum(in_x * weights))
    if prob > 0.5:
        return 1.0
    return 0.0
#data_mat_ce=np.array(data.iloc[20000:,:-1])
#for i in data_mat_ce[0:10]:
#  print(classify_vector(i,a))

##  4. 实际训练
###  4.1  随机梯度下降算法训练
*  使用平均值结果来衡量，由于训练数据划分没有随机随机化，结构是有些难受啊...
*  **但是对于数据的划分在改进之后会有很好的表现，我相信...**

In [10]:
def colic_test():
    # 使用 改进后的 随机梯度下降算法 求得在此数据集上的最佳回归系数 trainWeights
    # 训练数据  
    data_mat=np.array(data.iloc[:20000,:-1])
    class_labels=np.array(data.iloc[:20000,-1])
    # 计算回归系数
    train_weights = stoc_grad_ascent1(data_mat, class_labels, num_iter=500)
    
    # 初始化
    error_count = 0
    
    # 测试数据
    data_mat_ce=np.array(data.iloc[20000:,:-1])
    class_label_ce=np.array(data.iloc[20000:,-1])
    len_ce=len(class_label_ce)
    
    # 进行测试，计算分类错误的样本条数和最终的错误率
    for i in range(len_ce):
        if int(classify_vector(data_mat_ce[i],train_weights)) != int(class_label_ce[i]):
            error_count += 1
    error_rate = error_count / len_ce
    print('the error rate is {}'.format(error_rate))
    return error_rate
 
 
def multi_test():
    """
    调用 colicTest() 10次并求结果的平均值
    :return: nothing 
    """
    num_tests = 10
    error_sum = 0
    for k in range(num_tests):
        error_sum += colic_test()
    print('after {} iteration the average error rate is {}'.format(num_tests, error_sum / num_tests))
    
 
if __name__ == '__main__':
    #colic_test()
    multi_test()

the error rate is 0.2292014966961229
the error rate is 0.3648594857097365
the error rate is 0.2452034073720245
the error rate is 0.7559111535705756
the error rate is 0.7559111535705756
the error rate is 0.2309529496059231
the error rate is 0.22959955417562297
the error rate is 0.2239471379667224
the error rate is 0.2239471379667224
the error rate is 0.22784810126582278
after 10 iteration the average error rate is 0.34873815778998485
